In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /Users/acthach/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/acthach/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/acthach/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [3]:
whole_dataset = pd.read_csv('Input/Combined_News_DJIA.csv', header=0)
index = whole_dataset.loc[whole_dataset['Date'] == '2015-01-02'].index
num_of_rows = len(whole_dataset)

    #We add +1 to correct the zero-indexing
rows_to_keep = [*range(0, index[0]+1, 1)]

    #rows_to_keep = [whole_dataset[0:index[0]]]
df_train= pd.read_csv('Input/Combined_News_DJIA.csv', header=0, skiprows = lambda x: x not in rows_to_keep)
print( df_train)
df_test = pd.read_csv('Input/Combined_News_DJIA.csv', header=0, skiprows = lambda x: x in rows_to_keep)
print ( df_test)

            Date  Label                                               Top1  \
0     2008-08-08      0  b"Georgia 'downs two Russian warplanes' as cou...   
1     2008-08-11      1  b'Why wont America and Nato help us? If they w...   
2     2008-08-12      0  b'Remember that adorable 9-year-old who sang a...   
3     2008-08-13      0  b' U.S. refuses Israel weapons to attack Iran:...   
4     2008-08-14      1  b'All the experts admit that we should legalis...   
...          ...    ...                                                ...   
1606  2014-12-24      1  Death toll among Qatars 2022 World Cup workers...   
1607  2014-12-26      1  Saudis are eagerly awaiting the approval of a ...   
1608  2014-12-29      0  Solar Power Storage Prices Drop 25% In Germany...   
1609  2014-12-30      0  China businessman jailed for 13 years for buyi...   
1610  2014-12-31      0  AirAsia flight found at the bottom of the Java...   

                                                   Top2  \
0   